**French to Tamil Translation with Word2Vec**

In [ ]:
!pip install gensim gradio

In [ ]:
import gradio as gr
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch
from gensim.models import Word2Vec
import numpy as np

# Load pre-trained models and tokenizers
fr_en_tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fr-en")
fr_en_model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fr-en")

en_ta_tokenizer = AutoTokenizer.from_pretrained("suriya7/English-to-Tamil")
en_ta_model = AutoModelForSeq2SeqLM.from_pretrained("suriya7/English-to-Tamil")

# Load Word2Vec model (you should have a trained Word2Vec model saved)
# Assuming you have a trained Word2Vec model in your environment
# If not, you'll need to train it or load a pre-trained one.
# Here, I'm creating a sample Word2Vec model for demonstration purposes.
# Replace this with your actual Word2Vec model.

# Sample training data
sentences = [['bonjour', 'monde'], ['au revoir', 'monde'], ['salut', 'comment', 'ca', 'va']]
word2vec_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)

# Function to get similar words using Word2Vec
def get_similar_words(word, model, top_n=5):
    if word in model.wv:
        similar_words = model.wv.most_similar(word, topn=top_n)
        return [w[0] for w in similar_words]
    else:
        return []

# Function to translate French to English
def translate_fr_to_en(french_text):
    inputs = fr_en_tokenizer(french_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        translated_ids = fr_en_model.generate(**inputs)
    english_translation = fr_en_tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return english_translation

# Function to translate English to Tamil
def translate_en_to_ta(english_text):
    inputs = en_ta_tokenizer(english_text, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        translated_ids = en_ta_model.generate(**inputs)
    tamil_translation = en_ta_tokenizer.decode(translated_ids[0], skip_special_tokens=True)
    return tamil_translation

# Main function to handle translation from French to Tamil via English
def translate_french_to_tamil(french_word):
    english_translation = translate_fr_to_en(french_word)
    if english_translation.strip() == "":  # Check if translation is empty
        similar_words = get_similar_words(french_word, word2vec_model)
        if similar_words:
            # If no direct translation, use a similar word
            english_translation = translate_fr_to_en(similar_words[0])  # Use the first similar word
        else:
            return "No suitable translation found.", ""

    tamil_translation = translate_en_to_ta(english_translation)
    return english_translation, tamil_translation

# Create Gradio interface
iface = gr.Interface(fn=translate_french_to_tamil,
                     inputs="text",
                     outputs=["text", "text"],
                     title="French to Tamil Translation with Word2Vec",
                     description="Enter a French word to translate it to English and then to Tamil.")

# Launch the Gradio app
iface.launch()
